## 利用信息抽取技术搭建知识库

在这个notebook文件中，有些模板代码已经提供给你，但你还需要实现更多的功能来完成这个项目。除非有明确要求，你无须修改任何已给出的代码。以**'【练习】'**开始的标题表示接下来的代码部分中有你需要实现的功能。这些部分都配有详细的指导，需要实现的部分也会在注释中以'TODO'标出。请仔细阅读所有的提示。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

---

### 让我们开始吧

本项目的目的是结合命名实体识别、依存语法分析、实体消歧、实体统一对网站开放语料抓取的数据建立小型知识图谱。

在现实世界中，你需要拼凑一系列的模型来完成不同的任务；举个例子，用来预测狗种类的算法会与预测人类的算法不同。在做项目的过程中，你可能会遇到不少失败的预测，因为并不存在完美的算法和模型。你最终提交的不完美的解决方案也一定会给你带来一个有趣的学习经验！


---


## 步骤 1：实体统一

实体统一做的是对同一实体具有多个名称的情况进行统一，将多种称谓统一到一个实体上，并体现在实体的属性中（可以给实体建立“别称”属性）

例如：对“河北银行股份有限公司”、“河北银行公司”和“河北银行”我们都可以认为是一个实体，我们就可以将通过提取前两个称谓的主要内容，得到“河北银行”这个实体关键信息。

公司名称有其特点，例如后缀可以省略、上市公司的地名可以省略等等。在data/dict目录中提供了几个词典，可供实体统一使用。
- company_suffix.txt是公司的通用后缀词典
- company_business_scope.txt是公司经营范围常用词典
- co_Province_Dim.txt是省份词典
- co_City_Dim.txt是城市词典
- stopwords.txt是可供参考的停用词

### 练习1：
编写main_extract函数，实现对实体的名称提取“主体名称”的功能。

In [1]:
import jieba
import jieba.posseg as pseg
import re
import datetime


# 从输入的“公司名”中提取主体
def main_extract(input_str,stop_word,d_4_delete,d_city_province):
    # 开始分词并处理
    seg = pseg.cut(input_str)
    seg_lst = remove_word(seg,stop_word,d_4_delete)
    seg_lst = city_prov_ahead(seg_lst,d_city_province)
    return seg_lst

    
#TODO：实现公司名称中地名提前
def city_prov_ahead(seg_lst,d_city_province):
    city_prov_lst = []
    # TODO ...
    for word in seg_lst:
        if word.strip() in d_city_province and word.strip()!='':
            city_prov_lst.append(word.strip())
            seg_lst.remove(word.strip())
    

    return city_prov_lst+seg_lst




#TODO：替换特殊符号
def remove_word(seg,stop_word,d_4_delete):
    # TODO ...
    seg = list(seg)
    seg_lst = []
    for word,pos in seg:
        if word.strip() not in stop_word and word.strip() not in d_4_delete and word.strip()!='':
            seg_lst.append(word)
    
    return seg_lst


# 初始化，加载词典
def my_initial():
    fr1 = open(r"../data/dict/co_City_Dim.txt", encoding='utf-8')
    fr2 = open(r"../data/dict/co_Province_Dim.txt", encoding='utf-8')
    fr3 = open(r"../data/dict/company_business_scope.txt", encoding='utf-8')
    fr4 = open(r"../data/dict/company_suffix.txt", encoding='utf-8')
    #城市名
    lines1 = fr1.readlines()
    d_4_delete = []
    d_city_province = [re.sub(r'(\r|\n)*','',line) for line in lines1]
    #省份名
    lines2 = fr2.readlines()
    l2_tmp = [re.sub(r'(\r|\n)*','',line) for line in lines2]
    d_city_province.extend(l2_tmp)
    #公司后缀
    lines3 = fr3.readlines()
    l3_tmp = [re.sub(r'(\r|\n)*','',line) for line in lines3]
    lines4 = fr4.readlines()
    l4_tmp = [re.sub(r'(\r|\n)*','',line) for line in lines4]
    d_4_delete.extend(l4_tmp)
    #get stop_word
    fr = open(r'../data/dict/stopwords.txt', encoding='utf-8')   
    stop_word = fr.readlines()
    stop_word_after = [re.sub(r'(\r|\n)*','',stop_word[i]) for i in range(len(stop_word))]
    stop_word_after[-1] = stop_word[-1]
    stop_word = stop_word_after
    return d_4_delete,stop_word,d_city_province


In [2]:
# TODO：测试实体统一用例
d_4_delete,stop_word,d_city_province = my_initial()
company_name = "博美天津焊接材料有限公司"
lst = main_extract(company_name,stop_word,d_4_delete,d_city_province)
company_name = ''.join(lst)  # 对公司名提取主体部分，将包含相同主体部分的公司统一为一个实体
print(company_name)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/lx/h1cvbsnx1cq2_bnv5d7cltsh0000gn/T/jieba.cache
Loading model cost 0.667 seconds.
Prefix dict has been built succesfully.


天津博美焊接材料


## 步骤 2：实体识别
有很多开源工具可以帮助我们对实体进行识别。常见的有LTP、StanfordNLP、FoolNLTK等等。

本次采用FoolNLTK实现实体识别，fool是一个基于bi-lstm+CRF算法开发的深度学习开源NLP工具，包括了分词、实体识别等功能，大家可以通过fool很好地体会深度学习在该任务上的优缺点。

在‘data/train_data.csv’和‘data/test_data.csv’中是从网络上爬虫得到的上市公司公告，数据样例如下：

In [58]:
import pandas as pd
train_data = pd.read_csv('../data/info_extract/train_data.csv', encoding = 'gb2312', header=0)
train_data.head()

,id,sentence,tag,member1,member2
0,6461,与本公司关系:受同一公司控制 2，杭州富生电器有限公司企业类型: 有限公司注册地址: 富阳市...,0,0,0
1,2111,三、关联交易标的基本情况 1、交易标的基本情况 公司名称:红豆集团财务有限公司 公司地址:无...,0,0,0
2,9603,2016年协鑫集成科技股份有限公司向瑞峰（张家港）光伏科技有限公司支付设备款人民币4，515...,1,协鑫集成科技股份有限公司,瑞峰（张家港）光伏科技有限公司
3,3456,证券代码:600777 证券简称:新潮实业 公告编号:2015-091 烟台新潮实业股份有限...,0,0,0
4,8844,本集团及广发证券股份有限公司持有辽宁成大股份有限公司股票的本期变动系买卖一揽子沪深300指数...,1,广发证券股份有限公司,辽宁成大股份有限公司


In [59]:
test_data = pd.read_csv('../data/info_extract/test_data.csv', encoding = 'gb2312', header=0)
test_data.head()

,id,sentence
0,9259,2015年1月26日，多氟多化工股份有限公司与李云峰先生签署了《附条件生效的股份认购合同》
1,9136,2、2016年2月5日，深圳市新纶科技股份有限公司与侯毅先
2,220,2015年10月26日，山东华鹏玻璃股份有限公司与张德华先生签署了附条件生效条件的《股份认购合同》
3,9041,2、2015年12月31日，印纪娱乐传媒股份有限公司与肖文革签订了《印纪娱乐传媒股份有限公司...
4,10041,一、金发科技拟与熊海涛女士签订《股份转让协议》，协议约定：以每股1.0509元的收购价格，收...


我们选取一部分样本进行标注，即train_data，该数据由5列组成。id列表示原始样本序号；sentence列为我们截取的一段关键信息；如果关键信息中存在两个实体之间有股权交易关系则tag列为1，否则为0；如果tag为1，则在member1和member2列会记录两个实体出现在sentence中的名称。

剩下的样本没有标注，即test_data，该数据只有id和sentence两列，希望你能训练模型对test_data中的实体进行识别，并判断实体对之间有没有股权交易关系。

### 练习2：
将每句句子中实体识别出，存入实体词典，并用特殊符号替换语句。


In [60]:
import fool
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"
text = ["一个傻子在北京"]
print(fool.pos_cut(text))

[[('一个', 'm'), ('傻子', 'n'), ('在', 'p'), ('北京', 'ns')]]


In [61]:
# 处理test数据，利用开源工具进行实体识别和并使用实体统一函数存储实体

import fool
import pandas as pd
from copy import copy


test_data = pd.read_csv('../data/info_extract/test_data.csv', encoding = 'gb2312', header=0)
test_data['ner'] = None
ner_id = 1001
ner_dict_new = {}  # 存储所有实体
ner_dict_reverse_new = {}  # 存储所有实体

for i in range(len(test_data)):
    sentence = copy(test_data.iloc[i, 1])

    # TODO：调用fool进行实体识别，得到words和ners结果
    # TODO ...
    words, ners = fool.analysis(sentence)
    
    ners[0].sort(key=lambda x:x[0], reverse=True)
    for start, end, ner_type, ner_name in ners[0]:
        if ner_type=='company' or ner_type=='person':
            # TODO：调用实体统一函数，存储统一后的实体
            # 并自增ner_id
            # TODO ...
            lst = main_extract(ner_name,stop_word,d_4_delete,d_city_province)
            company_main_name = ''.join(lst)
            print(company_main_name)
            ner_dict_new[company_main_name] = ner_id
            ner_id += 1
            
            # 在句子中用编号替换实体名
            sentence = sentence[:start] + ' ner_' + str(ner_dict_new[company_main_name]) + '_ ' + sentence[end:]
    test_data.iloc[i, -1] = sentence

X_test = test_data[['ner']]


李云峰
氟化工
侯毅
深圳市新纶科技股份
张德华
山东华鹏玻璃
肖文革
印纪娱乐传媒
熊海涛
金发科技
宋琳
上海新朋实业
康力电梯
王友林
神州易桥100股权
彭聪
曹飞
百川能源
颜军
珠海欧比特控制工程
宋睿
成都云图
柯塞威
上海岩石企业
邓冠华
广州阳普医疗科技股份
金固
孙锋峰
李云峰
李云峰
氟化工
曹飞
百川能源
林奇
游族网络
江斌
厦门金达威集团股份
王友林
康力电梯
林海峰
东方日升新能源
云科服务
郭为
神州数码集团股份
吴宏亮
浙江唐德影视
灵云传媒
王利平
广博
周旭辉
金亚科技
天象互动
成都天象互动科技
周旭辉
金亚科技
金亚科技股份
汉鼎宇佑互联网
吴艳
浙江奥鑫
杭州雄健投资合伙企业
天津和达股权投资基金管理合伙企业
深圳市方德智联投资管理
吴艳
汉鼎宇佑互联网
衡胜投资
杜应流
邓冠华
广州阳普医疗科技股份
江苏亨通光电
崔根良
周福海
江苏亚太轻合金科技股份
谭颂斌
广东银禧科技股份
赵东日
山东日科化学
钟烈华
广东塔牌集团股份
郭金东
金浦钛业
郭金东
金浦钛业
金浦钛业
郭金东
西水股份
西水股份
中国电子财务有限责任
南京熊猫电子
腾达建设集团股份
腾达建设
东投能源投资
上海申华风电新能源
西安大雁塔支行
中国银行
远东
红豆集团
金地商置
深圳威新软件科技
广发银行
淮安广发银行分行
锐电国际贸易
锐电设计
泰格林纸
泰格林纸集团股份
大唐塔山发电有限责任
大唐国际发电
广发证券
广发证券
中航资本
中航工业
刘梦龙
深圳市易尚展示
北京市盈科律师
国盛证券有限责任
沈阳和平支行
中国工商银行
广州中石化大厦支行
平安银行
孚支行
浙江南浔农村商业银行
亿阳信通
亿阳集团股份
维科
宁波维科精华集团股份
柯罗尼
誉德能源
开封汽博园
开封申华汽车博展
沈金彦
沈金彦
中国信息信托投资公司
中国电子财务有限责任
宁夏隆基
宁夏隆基
上海中国证券登记结算有限责任
上海中国证券登记结算有限责任
深圳市梅江南投资
深圳市梅江南投资
上海实
上海国际
博苑公司
天津博苑高新材料
中国电力建设
中国电建
王忠岩
辽宁时代万恒
钱江四桥
杭州钱江四桥经营
温州民商银行
温州民商银行
牡丹江恒丰纸业有限责任
牡丹江恒丰纸业
广东四通集团
四通
药业
五洲松德联合会计师
康美健康保险
康美保险
广州珠江新城支行
广发银行
天津金岸重

In [64]:
# 处理train数据，利用开源工具进行实体识别和并使用实体统一函数存储实体
train_data = pd.read_csv('../data/info_extract/train_data.csv', encoding = 'gb2312', header=0)
train_data['ner'] = None

for i in range(len(train_data)):
    # 判断正负样本
    if train_data.iloc[i,:]['member1']=='0' and train_data.iloc[i,:]['member2']=='0':
        sentence = copy(train_data.iloc[i, 1])
        # TODO：调用fool进行实体识别，得到words和ners结果
        # TODO ...
        words, ners = fool.analysis(sentence)

        ners[0].sort(key=lambda x:x[0], reverse=True)
        for start, end, ner_type, ner_name in ners[0]:
            if ner_type=='company' or ner_type=='person':
                # TODO：调用实体统一函数，存储统一后的实体
                # 并自增ner_id
                # TODO ...
                lst = main_extract(ner_name,stop_word,d_4_delete,d_city_province)
                company_main_name = ''.join(lst)
                ner_dict_new[company_main_name] = ner_id
                ner_id += 1

                # 在句子中用编号替换实体名
                sentence = sentence[:start] + ' ner_' + str(ner_dict_new[company_main_name]) + '_ ' + sentence[end:]
        train_data.iloc[i, -1] = sentence
    else:
        # 将训练集中正样本已经标注的实体也使用编码替换
        sentence = copy(train_data.iloc[i,:]['sentence'])
        for company_main_name in [train_data.iloc[i,:]['member1'],train_data.iloc[i,:]['member2']]:
            # TODO：调用实体统一函数，存储统一后的实体
            # 并自增ner_id
            # TODO ...
            lst = main_extract(company_main_name,stop_word,d_4_delete,d_city_province)
            new_company_main_name = ''.join(lst)
            ner_dict_new[new_company_main_name] = ner_id
            ner_id+=1

            # 在句子中用编号替换实体名
            sentence = re.sub(company_main_name, ' ner_%s_ '%(str(ner_dict_new[new_company_main_name])), sentence)
        train_data.iloc[i, -1] = sentence
        
y = train_data.loc[:,['tag']]
train_num = len(train_data)
X_train = train_data[['ner']]

# 将train和test放在一起提取特征
X = pd.concat([X_train, X_test])


In [65]:
X_train

,ner
0,与本公司关系:受同一公司控制 2， ner_1854_ 企业类型: 有限公司注册地址: 富阳...
1,三、关联交易标的基本情况 1、交易标的基本情况 公司名称: ner_1856_ 地址:无锡市...
2,2016年 ner_1857_ 向 ner_1858_ 支付设备款人民币4，515，770.00元
3,证券代码:600777 证券简称: ner_1860_ 公告编号:2015-091 ner...
4,本集团及 ner_1861_ 持有 ner_1862_ 股票的本期变动系买卖一揽子沪深300...
...,...
845,请 你公司结合 ner_3541_ 的经营变化状况、上述增资价格与本次交易作价的差异，进一步...
846,公司使用暂时闲置募集资金6，483万元购买了 ner_3543_ ner_3542_ “本...
847,(二) 本次重组尚需履行的决策程序18 本次交易尚需履行的决策和审批程序包括但不限于:1、本...
848,2015 年 5 月 6 日，公司以人民币 3，000 万元闲置募集资金购买了 ner_35...


In [66]:
X_test

,ner
0,2015年1月26日， ner_1002_ 与 ner_1001_ 先生签署了《附条件生效的...
1,2、2016年2月5日， ner_1004_ 与 ner_1003_ 先
2,2015年10月26日， ner_1006_ 与 ner_1005_ 先生签署了附条件生效条...
3,2、2015年12月31日， ner_1008_ 与 ner_1007_ 签订了《印纪娱乐传...
4,一、 ner_1010_ 拟与 ner_1009_ 女士签订《股份转让协议》，协议约定：以每...
...,...
414,近日，该子公司已完成工商注册登记手续，并领取了南京市工商行政管理局颁发的<企业法人营业执照>...
415,(二)本次交易构成关联交易正元投资拟认购金额不低于 13 亿元且不低于本次配套融资总额的 2...
416,证券代码:600225 证券简称: ner_1848_ 公告编号:临 2015-118 n...
417,2015年3月31日， ner_1850_ 与 ner_1849_ 签署了附条件生效的《湖南...


In [67]:
X

,ner
0,与本公司关系:受同一公司控制 2， ner_1854_ 企业类型: 有限公司注册地址: 富阳...
1,三、关联交易标的基本情况 1、交易标的基本情况 公司名称: ner_1856_ 地址:无锡市...
2,2016年 ner_1857_ 向 ner_1858_ 支付设备款人民币4，515，770.00元
3,证券代码:600777 证券简称: ner_1860_ 公告编号:2015-091 ner...
4,本集团及 ner_1861_ 持有 ner_1862_ 股票的本期变动系买卖一揽子沪深300...
...,...
414,近日，该子公司已完成工商注册登记手续，并领取了南京市工商行政管理局颁发的<企业法人营业执照>...
415,(二)本次交易构成关联交易正元投资拟认购金额不低于 13 亿元且不低于本次配套融资总额的 2...
416,证券代码:600225 证券简称: ner_1848_ 公告编号:临 2015-118 n...
417,2015年3月31日， ner_1850_ 与 ner_1849_ 签署了附条件生效的《湖南...


In [69]:
ner_dict_new

{'李云峰': 1030,
 '氟化工': 1031,
 '侯毅': 1003,
 '深圳市新纶科技股份': 1004,
 '张德华': 1933,
 '山东华鹏玻璃': 2235,
 '肖文革': 1007,
 '印纪娱乐传媒': 1008,
 '熊海涛': 1692,
 '金发科技': 1691,
 '宋琳': 1011,
 '上海新朋实业': 1012,
 '康力电梯': 1039,
 '王友林': 1038,
 '神州易桥100股权': 1015,
 '彭聪': 1016,
 '曹飞': 1032,
 '百川能源': 1033,
 '颜军': 1019,
 '珠海欧比特控制工程': 1020,
 '宋睿': 1021,
 '成都云图': 1022,
 '柯塞威': 1023,
 '上海岩石企业': 1024,
 '邓冠华': 1067,
 '广州阳普医疗科技股份': 1068,
 '金固': 1027,
 '孙锋峰': 1028,
 '林奇': 1034,
 '游族网络': 1035,
 '江斌': 1036,
 '厦门金达威集团股份': 1037,
 '林海峰': 1040,
 '东方日升新能源': 1041,
 '云科服务': 1042,
 '郭为': 1043,
 '神州数码集团股份': 1044,
 '吴宏亮': 1045,
 '浙江唐德影视': 1046,
 '灵云传媒': 1047,
 '王利平': 1048,
 '广博': 1049,
 '周旭辉': 1054,
 '金亚科技': 1055,
 '天象互动': 1052,
 '成都天象互动科技': 1053,
 '金亚科技股份': 1056,
 '汉鼎宇佑互联网': 3024,
 '吴艳': 1063,
 '浙江奥鑫': 1059,
 '杭州雄健投资合伙企业': 1060,
 '天津和达股权投资基金管理合伙企业': 1061,
 '深圳市方德智联投资管理': 1062,
 '衡胜投资': 1065,
 '杜应流': 1066,
 '江苏亨通光电': 1069,
 '崔根良': 1070,
 '周福海': 1071,
 '江苏亚太轻合金科技股份': 1072,
 '谭颂斌': 1213,
 '广东银禧科技股份': 1214,
 '赵东日': 1075,
 '山东日科化学': 2402,
 '钟烈华

## 步骤 3：关系抽取


目标：借助句法分析工具，和实体识别的结果，以及文本特征，基于训练数据抽取关系，并存储进图数据库。

本次要求抽取股权交易关系，关系为无向边，不要求判断投资方和被投资方，只要求得到双方是否存在交易关系。

模板建立可以使用“正则表达式”、“实体间距离”、“实体上下文”、“依存句法”等。

答案提交在submit目录中，命名为info_extract_submit.csv和info_extract_entity.csv。
- info_extract_entity.csv格式为：第一列是实体编号，第二列是实体名（实体统一的多个实体名用“|”分隔）
- info_extract_submit.csv格式为：第一列是关系中实体1的编号，第二列为关系中实体2的编号。

示例：
- info_extract_entity.csv

| 实体编号 | 实体名 |
| ------ | ------ |
| 1001 | 小王 |
| 1002 | A化工厂 |

- info_extract_submit.csv

| 实体1 | 实体2 |
| ------ | ------ |
| 1001 | 1003 |
| 1002 | 1001 |

### 练习3：提取文本tf-idf特征

去除停用词，并转换成tfidf向量。

In [70]:
from pyltp import Segmentor

In [71]:
with open('../data/user_dict.txt', 'w') as fw:
    fw.write('ner\_1001\_')
# 初始化实例
segmentor = Segmentor()  
# 加载模型，加载自定义词典
segmentor.load_with_lexicon('../data/ltp_data_v3.4.0/cws.model', '../data/user_dict.txt') 
print(list(segmentor.segment('ner_1001_')))

['ner', '_', '1001_']


In [72]:
# encoding=utf-8
import jieba

# 基于jieba的分词
jieba.add_word('ner_1001_')
seg_list = jieba.cut("ner_1001_收购ner_1002_", cut_all=False)

list(seg_list)



['ner_1001_', '收购', 'ner_1002_']

In [ ]:
# code
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
from pyltp import Segmentor


# 实体符号加入分词词典
with open('../data/user_dict.txt', 'w') as fw:
    for v in ['一', '二', '三', '四', '五', '六', '七', '八', '九', '十']:
        fw.write( v + '号企业 ni\n')

# 初始化实例
segmentor = Segmentor()  
# 加载模型，加载自定义词典
segmentor.load_with_lexicon('../data/ltp_data_v3.4.0/cws.model', '../data/user_dict.txt')  

# 加载停用词
fr = open(r'../data/dict/stopwords.txt', encoding='utf-8')   
stop_word = fr.readlines()
stop_word = [re.sub(r'(\r|\n)*','',stop_word[i]) for i in range(len(stop_word))]



# # 分词
# f = lambda x: ' '.join([for word in segmentor.segment(x) if word not in stop_word and not re.findall(r'ner\_\d\d\d\d\_', word)])
# corpus=X['ner'].map(f).tolist()

corpus = []
for ner in X['ner']:
    f = []
  #  print(list(segmentor.segment(ner)))
    for word in segmentor.segment(ner):
        if word not in stop_word and not re.findall(r'ner_\d\d\d\d\_', word):
            f.append(word)
    corpus.append(' '.join(f))

print(corpus)









In [73]:
# code
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
from pyltp import Segmentor


# 实体符号加入分词词典

for v in ['一号企业', '二号企业', '三号企业', '四号企业', '五号企业', '六号企业', '七号企业', '八号企业', '九号企业', '十号企业']:
    jieba.add_word(v)

for i in range(1000,10000):
    s = 'ner_'+str(i)+'_'
    jieba.add_word(s)


# 加载停用词
fr = open(r'../data/dict/stopwords.txt', encoding='utf-8')   
stop_word = fr.readlines()
stop_word = [re.sub(r'(\r|\n)*','',stop_word[i]) for i in range(len(stop_word))]



# # 分词
# f = lambda x: ' '.join([for word in segmentor.segment(x) if word not in stop_word and not re.findall(r'ner\_\d\d\d\d\_', word)])
# corpus=X['ner'].map(f).tolist()

corpus = []
for ner in X['ner']:
    f = []
  #  print(list(segmentor.segment(ner)))
    seg_list = jieba.cut(ner, cut_all=False)
    for word in seg_list:
        if word.strip() not in stop_word and word.strip()!='':
            f.append(word.strip())
    corpus.append(' '.join(f))

print(len(corpus[:len(X_train)]))









850


In [74]:
corpus

['公司 受 同一 公司 控制 2 ner_1854_ 企业 类型 有限公司 注册 地址 富阳市 东洲 街道 东洲 工业 功能区 九号 路 1 号 法定代表 ner_1853_ 注册资本 16 000 万元 经营范围 许可 经营项目 制造 高效 节能 感应 电机 ; 普通 货运',
 '三 关联 交易 标的 1 交易 标的 公司 名称 ner_1856_ 地址 无锡市 锡山区 东港 镇锡港 东路 2 号 企业 类型 有限责任 公司 法定代表 ner_1855_ 注册资本 50000 万元 整 财务 公司 公司 参股 公司 成立 2008 11 经营范围 成员 单位 财务 融资 顾问 信用 鉴证 咨询 代理 业务 ; 成员 单位 交 易 款项 收付 ; 批准 保险代理 业务 ; 成员 单位 担保 ; 成员 单位 间 委托 贷款 委托 投资 ; 成员 单位 票据 承兑 贴现 ; 成员 单位 转账 结算 结算 清算 方案设计 ; 吸收 成员 单位 存款 ; 成员 单 位 贷款 融资 租赁 ; 同业 拆借 ; 承销 成员 单位 企业债券 ; 金融机构 股权 投资 ; 有价证券 投资 股票 二级 市场 投资 除外 ; 成员 单位 产品 买方 信 贷',
 '2016 ner_1857_ ner_1858_ 支付 设备 款 人民币 4 515 770.00 元',
 '证券 代码 600777 证券 简称 ner_1860_ 公告 编号 2015 091 ner_1859_ 第八届 监事会 第十一次 会议 决议 公告 公司 监事会 全体 监事 公告 不 虚假 记载 误导性 陈述 遗漏 真实性 准确性 完整性 承担 个别 连带责任',
 '集团 ner_1861_ 持有 ner_1862_ 股票 本期 变动 系 买卖 一揽子 沪 深 300 指数 成份股 所致',
 '二 被担保人 被担保人 ner_1864_ 住所 上海市 杨浦区 中山 北二路 1121 号 10 楼 法定代表 ner_1863_ 提示 公司 董事会 全体 董事 公告 不 虚假 记载 误导性 陈述 遗漏 真实性 准确性 完整性 承担 个别 连带责任',
 '2014 12 12 日 绵阳市 国资委 转让 ner_1866_ 股权 批复 ner_1865_ 2014 82 号 公司 出售 国虹 通讯 股权',
 

In [75]:

from sklearn.feature_extraction.text import TfidfVectorizer
# TODO：提取tfidf特征
# TODO ...
X_train_new = corpus[:len(X_train)]
X_test_new = corpus[len(X_train):]

vectorizer = TfidfVectorizer()
X_train_features = vectorizer.fit_transform(X_train_new)
X_test_features = vectorizer.transform(X_test_new)


X_train_features.toarray()




array([[0.        , 0.09956024, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.25610691, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.12464233, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15427629, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### 练习4：提取句法特征
除了词语层面的句向量特征，我们还可以从句法入手，提取一些句法分析的特征。

参考特征：

1、企业实体间距离

2、企业实体间句法距离

3、企业实体分别和关键触发词的距离

4、实体的依存关系类别

In [76]:
# -*- coding: utf-8 -*-
from pyltp import Parser
from pyltp import Segmentor
from pyltp import Postagger
import networkx as nx
import matplotlib.pyplot as plt
from pylab import mpl
import re
import numpy as np

postagger = Postagger() # 初始化实例
postagger.load_with_lexicon('../data/ltp_data_v3.4.0/pos.model', '../data/user_dict.txt')  # 加载模型
segmentor = Segmentor()  # 初始化实例
segmentor.load_with_lexicon('../data/ltp_data_v3.4.0/cws.model', '../data/user_dict.txt')  # 加载模型



def parse(s):
    """
    对语句进行句法分析，并返回句法结果
    """
    tmp_ner_dict = {}
    aa=s
    num_lst = ['一', '二', '三', '四', '五', '六', '七', '八', '九', '十']
    dr_list = ['SBV','VOB','IOB','FOB','DBL','ATT','ADV','CMP','COO','POB','LAD','RAD','IS','HED']

    # 将公司代码替换为特殊称谓，保证分词词性正确
    for i, ner in enumerate(list(set(re.findall(r'(ner\_\d\d\d\d\_)', s)))):
        try:
            tmp_ner_dict[num_lst[i]+'号企业'] = ner
        except IndexError:
            # TODO：定义错误情况的输出
            # TODO ...
            print("错误")
            
        s = s.replace(ner, num_lst[i]+'号企业')

    words = segmentor.segment(s)
    tags = postagger.postag(words)
    parser = Parser() # 初始化实例
    parser.load('../data/ltp_data_v3.4.0/parser.model')  # 加载模型
    arcs = parser.parse(words, tags)  # 句法分析
    arcs_lst = list(map(list, zip(*[[arc.head, arc.relation] for arc in arcs])))
    
    # 句法分析结果输出
    parse_result = pd.DataFrame([[a,b,c,d] for a,b,c,d in zip(list(words),list(tags), arcs_lst[0], arcs_lst[1])], index = range(1,len(words)+1))
    parser.release()  # 释放模型
    

    grammar = np.zeros(len(dr_list))
    
    # TODO：提取企业实体依存句法类型
    # TODO ...    
    for i in range(len(parse_result)):
        if parse_result.iloc[i][0] in tmp_ner_dict:
            index = dr_list.index(parse_result.iloc[i][3])
            grammar[index]+=1
        
    
#     rely_id = [arc.head for arc in arcs] 
#     print(rely_id)
#     relation = [arc.relation for arc in arcs]  # 提取依存关系
#     heads = ['Root' if id == 0 else words[id-1] for id in rely_id]  # 匹配依存父节点词语
#     print(heads)

#     for i in range(len(words)):
#         print(relation[i] + '(' + words[i] + ', ' + heads[i] + ')')
    

    

    # 投资关系关键词
    key_words = ["收购","竞拍","转让","扩张","并购","注资","整合","并入","竞购","竞买","支付","收购价","收购价格","承购","购得","购进",
             "购入","买进","买入","赎买","购销","议购","函购","函售","抛售","售卖","销售","转售"]
    # TODO：*根据关键词和对应句法关系提取特征（如没有思路可以不完成）
    # TODO ...
    if '一号企业' not in list(words) or '二号企业' not in list(words):
        print(list(words))
    
    index1 = list(words).index('一号企业') + 1
    index2 = list(words).index('二号企业') + 1

    

    d1 = abs(index1-index2)
    d2 = shortest_path(parse_result, index1, index2 )
    
    d3 = np.zeros(len(key_words))
    d4 = np.zeros(len(key_words))
    for word in list(words):
        if word in key_words:
            index3 = list(words).index(word) + 1
            d3[index3] = shortest_path(parse_result,index1,index3)
            d4[index3] = shortest_path(parse_result,index2,index3)
    
    feature = np.concatenate((grammar,[d1,d2],d3,d4))
    
    parser.release()  # 释放模型
    return feature


def shortest_path(arcs_ret, source, target):
    """
    求出两个词最短依存句法路径，不存在路径返回-1
    arcs_ret：句法分析结果
    source：实体1
    target：实体2
    """
    G=nx.DiGraph()
    # 为这个网络添加节点...
    for i in list(arcs_ret.index):
        G.add_node(i)
    # TODO：在网络中添加带权中的边...（注意，我们需要的是无向边）
    # TODO ...
    G.add_node(0)
    
    heads = list(arcs_ret[2])
    
    for i in range(len(heads)):
        G.add_edge(i+1, heads[i])
        G.add_edge(heads[i],i+1)
    
    
    
#     plt.figure(figsize=(8, 8))
#     nx.draw(G, node_color="blue", with_labels=True)
#     plt.axis('equal')
#     plt.show()
    
    

    try:
        # TODO：利用nx包中shortest_path_length方法实现最短距离提取
        # TODO ...
        distance = nx.shortest_path_length(G, source=source, target=target)
        return distance
    except:
        return -1


def get_feature(s):
    """
    汇总上述函数汇总句法分析特征与TFIDF特征
    """
    # TODO：汇总上述函数汇总句法分析特征与TFIDF特征
    # TODO ...
    
    features = vectorizer.transform([s]).toarray()
    grammar_feature = parse(s)
    features = np.concatenate((features[0],grammar_feature))
    return features


In [54]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(len(X_train['ner'])):
    s = X_train['ner'][i]
    label = train_data['tag'][i]
    feature = get_feature(s)
    x_train.append(feature)
    y_train.append(label)
x_train


['本', '年度', '上海', '一号企业', '收购', '天津', '一号企业', '焊接', '材料', '有限公司', '将', '其', '所', '持', '天津', '一号企业', '销售', '44%', '股权', '，', '支付', '对', '价8', '，', '730', '，', '480.00', '元']


ValueError: '二号企业' is not in list

### 练习5：建立分类器

利用已经提取好的tfidf特征以及parse特征，建立分类器进行分类任务。

In [51]:
# 建立分类器进行分类
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# TODO：定义需要遍历的参数

parameters = {'C':[0.00001, 0.0001, 0.001, 0.005,0.01,0.05, 0.1, 0.5,1,2,5,10]}



# TODO：选择模型
lr = LogisticRegression()


# TODO：利用GridSearchCV搜索最佳参数
lr.fit(X_train, y_train).score(X_test, y_test)

clf = GridSearchCV(lr, parameters, cv=10,n_jobs=10)

clf.fit(X_train, y_train)
clf.score(X_test, y_test)
print (clf.best_params_)

# TODO：对Test_data进行分类



# TODO：保存Test_data分类结果
# 答案提交在submit目录中，命名为info_extract_submit.csv和info_extract_entity.csv。
# info_extract_entity.csv格式为：第一列是实体编号，第二列是实体名（实体统一的多个实体名用“|”分隔）
# info_extract_submit.csv格式为：第一列是关系中实体1的编号，第二列为关系中实体2的编号。

 

### 练习6：操作图数据库
对关系最好的描述就是用图，那这里就需要使用图数据库，目前最常用的图数据库是noe4j，通过cypher语句就可以操作图数据库的增删改查。可以参考“https://cuiqingcai.com/4778.html”。

本次作业我们使用neo4j作为图数据库，neo4j需要java环境，请先配置好环境。

将我们提出的实体关系插入图数据库，并查询某节点的3层投资关系，即三个节点组成的路径（如果有的话）。如果无法找到3层投资关系，请查询出任意指定节点的投资路径。

In [ ]:

from py2neo import Node, Relationship, Graph

graph = Graph(
    "http://localhost:7474", 
    username="neo4j", 
    password="person"
)

for v in relation_list:
    a = Node('Company', name=v[0])
    b = Node('Company', name=v[1])
    
    # 本次不区分投资方和被投资方，无向图
    r = Relationship(a, 'INVEST', b)
    s = a | b | r
    graph.create(s)
    r = Relationship(b, 'INVEST', a)
    s = a | b | r
    graph.create(s)

In [ ]:
# TODO：查询某节点的3层投资关系


## 步骤4：实体消歧
解决了实体识别和关系的提取，我们已经完成了一大截，但是我们提取的实体究竟对应知识库中哪个实体呢？下图中，光是“苹果”就对应了13个同名实体。
<img src="../image/baike2.png", width=340, heigth=480>

在这个问题上，实体消歧旨在解决文本中广泛存在的名称歧义问题，将句中识别的实体与知识库中实体进行匹配，解决实体歧义问题。


### 练习7：
匹配test_data.csv中前25条样本中的人物实体对应的百度百科URL（此部分样本中所有人名均可在百度百科中链接到）。

利用scrapy、beautifulsoup、request等python包对百度百科进行爬虫，判断是否具有一词多义的情况，如果有的话，选择最佳实体进行匹配。

使用URL为‘https://baike.baidu.com/item/’+人名 可以访问百度百科该人名的词条，此处需要根据爬取到的网页识别该词条是否对应多个实体，如下图：
<img src="../image/baike1.png", width=440, heigth=480>
如果该词条有对应多个实体，请返回正确匹配的实体URL，例如该示例网页中的‘https://baike.baidu.com/item/陆永/20793929’。

- 提交文件：entity_disambiguation_submit.csv
- 提交格式：第一列为实体id（与info_extract_submit.csv中id保持一致），第二列为对应URL。
- 示例：

| 实体编号 | URL |
| ------ | ------ |
| 1001 | https://baike.baidu.com/item/陆永/20793929 |
| 1002 | https://baike.baidu.com/item/王芳/567232 |


In [ ]:
import jieba
import pandas as pd

# 找出test_data.csv中前25条样本所有的人物名称，以及人物所在文档的上下文内容
test_data = pd.read_csv('../data/info_extract/test_data.csv', encoding = 'gb2312', header=0)

# 存储人物以及上下文信息（key为人物ID，value为人物名称、人物上下文内容）
person_name = {}

# 观察上下文的窗口大小
window = 10  

# 遍历前25条样本
for i in range(25):
    sentence = copy(test_data.iloc[i, 1])
    words, ners = fool.analysis(sentence)
    ners[0].sort(key=lambda x:x[0], reverse=True)
    for start, end, ner_type, ner_name in ners[0]:
        if ner_type=='person':
            # TODO：提取实体的上下文
            



# 利用爬虫得到每个人物名称对应的URL
# TODO：找到每个人物实体的词条内容。

# TODO：将样本中人物上下文与爬取词条结果进行对比，选择最接近的词条。



# 输出结果
pd.DataFrame(result_data).to_csv('../submit/entity_disambiguation_submit.csv', index=False)
